<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

#### - Función de callback definida en clase

In [19]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### Desafío 2 - Consignas

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

## Resolución del desafío

### Ejercicio 
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.


#### Carga del dataset

In [2]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

/home/martin/Documents/CEIA/CEIA-NLP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Elijo como dataset las novelas de Harry Potter disponibles Hugging Face


# Load dataset from HF
dataset = load_dataset("elricwan/HarryPotter", split="train")

# Convert to pandas DataFrame if needed
df_HarryPotter = dataset.to_pandas()

print(df_HarryPotter.head())

                                          filename  \
0      1-Harry-Potter-and-the-Sorcerer’s-Stone.txt   
1    2-Harry-Potter-and-the-Chamber-of-Secrets.txt   
2   3-Harry-Potter-and-the-Prisoner-of-Azkaban.txt   
3        4-Harry-Potter-and-the-Goblet-of-Fire.txt   
4  5-Harry-Potter-and-the-Order-of-the-Phoenix.txt   

                                             content  
0  FOR JESSICA, WHO LOVES STORIES,\n\nFOR ANNE, W...  
1  FOR SEÁN P. F. HARRIS,\n\nGETAWAY DRIVER AND F...  
2  TO JILL PREWETT AND\n\nAINE KIELY,\n\nTHE GODM...  
3  TO PETER ROWLING,\n\nIN MEMORY OF MR. RIDLEY\n...  
4  TO NEIL, JESSICA, AND DAVID,\n\nWHO MAKE MY WO...  


In [4]:
print("Cantidad de documentos:", df_HarryPotter.shape[0])
print("Nombre de las columnas del DF: ", df_HarryPotter.columns)


Cantidad de documentos: 8
Nombre de las columnas del DF:  Index(['filename', 'content'], dtype='object')


## Observaciones:
El dataset presenta 8 archivos de texto correspondientes a los 8 libros de Harry Potter.
Debido a las diferencias con el dataset visto en clase, se utilizarán las siguientes definiciones:
 1) Se concatenarán todos los libros para formar el corpus.
 2) Se considerarán a los párrafos como los documentos para realizar los embeddings.

In [7]:
import re
import nltk

#nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Remueve puntuación, convierte a minúsculas, elimina stopwords y tokeniza el texto.
    
    """
    # Lowercase
    text = text.lower()
    
    # Keep only alphabetic characters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Split into words
    tokens = text.split()
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens


In [9]:
# Verificar los documentos, y si se puede dividirlos en párrafos
sentence_tokens_hp = []

for book in df_HarryPotter['content']:
    paragraphs = book.split("\n")  # Dividir el texto en párrafos usando el salto de línea como separador
    for paragraph in paragraphs:
        tokens = preprocess_text(paragraph)
        if tokens:  # Solo agregar si no está vacío
            sentence_tokens_hp.append(tokens)

print(f"Total 'documents': {len(sentence_tokens_hp)}")
print("Primer doc:", sentence_tokens_hp[0][:50])


Total 'documents': 80066
Primer doc: ['jessica', 'loves', 'stories']


#Analisis del impacto de limpiar el dataset:

In [10]:
#Analizar la cantidad de tokens en el texto original vs luego de aplicar la funcion de procesar el texto
raw_tokens = [word for sentence in df_HarryPotter['content'] for word in sentence.split()]
clean_tokens = [word for sentence in sentence_tokens_hp for word in sentence]

print("Tokens únicos (en texto completo):", len(set(raw_tokens)))
print("Tokens únicos (en texto limpio):", len(set(clean_tokens)))


Tokens únicos (en texto completo): 58427
Tokens únicos (en texto limpio): 23018


In [11]:
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

raw_tokens_norm = [re.sub(r'[^a-z]', '', w.lower()) for w in raw_tokens if w.strip() != '']

stop_before = sum(1 for w in raw_tokens if w.lower() in stop_words)
stop_after  = sum(1 for w in clean_tokens if w.lower() in stop_words)
stop_before_norm = sum(1 for w in raw_tokens_norm if w in stop_words)

print("Stopwords antes:", stop_before)
print("Stopwords después:", stop_after)
print("Stopwords normalizados después:", stop_after)


[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Stopwords antes: 916922
Stopwords después: 0
Stopwords normalizados después: 0


In [12]:
# Aplicar el preprocesamiento a cada parragrafo
# 1) Concatenar todo el texto
all_text = " ".join(df_HarryPotter['content'].tolist())

# Separar por párrafos
paragraphs = all_text.split("\n\n")

# Preprocesar cada párrafo
sentence_tokens_hp = [preprocess_text(p) for p in paragraphs if p.strip()]

print("Ejemplo de párrafo tokenizado:", sentence_tokens_hp[0][:30])
print("Total de párrafos procesados:", len(sentence_tokens_hp))


Ejemplo de párrafo tokenizado: ['jessica', 'loves', 'stories']
Total de párrafos procesados: 80570


In [13]:
from gensim.models import Word2Vec

model_hp_potter = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=1 # modelo 0:CBOW  1:skipgram
)
model_hp_potter_cbow = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=0 # modelo 0:CBOW  1:skipgram
)

In [14]:
# Obtener el vocabulario con los tokens
model_hp_potter.build_vocab(sentence_tokens_hp)

In [15]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", model_hp_potter.corpus_count)

Cantidad de docs en el corpus: 80570


In [16]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(model_hp_potter.wv.index_to_key))

Cantidad de words distintas en el corpus: 12352


In [20]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
model_hp_potter.train(sentence_tokens_hp,
                 total_examples=model_hp_potter.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3404021.75
Loss after epoch 1: 2876884.75
Loss after epoch 2: 2596651.5
Loss after epoch 3: 2356467.0
Loss after epoch 4: 2347464.0
Loss after epoch 5: 2340265.0
Loss after epoch 6: 2082078.0
Loss after epoch 7: 1928440.0
Loss after epoch 8: 1913188.0
Loss after epoch 9: 1906228.0
Loss after epoch 10: 1891436.0
Loss after epoch 11: 1881448.0
Loss after epoch 12: 1897572.0
Loss after epoch 13: 1883528.0
Loss after epoch 14: 1878820.0
Loss after epoch 15: 1183612.0
Loss after epoch 16: 1044196.0
Loss after epoch 17: 1047176.0
Loss after epoch 18: 997840.0
Loss after epoch 19: 1030496.0
Loss after epoch 20: 1001608.0
Loss after epoch 21: 1015496.0
Loss after epoch 22: 1016884.0
Loss after epoch 23: 1011548.0
Loss after epoch 24: 1002792.0
Loss after epoch 25: 1001144.0
Loss after epoch 26: 971816.0
Loss after epoch 27: 993688.0
Loss after epoch 28: 983384.0
Loss after epoch 29: 975816.0
Loss after epoch 30: 982184.0
Loss after epoch 31: 978040.0
Loss after epoch 32: 95

(55164765, 60602400)

In [21]:
# Entrenamos ahora el modelo CBOW
model_hp_potter_cbow.train(sentence_tokens_hp,
                 total_examples=model_hp_potter_cbow.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 586162.375
Loss after epoch 1: 552943.0
Loss after epoch 2: 522120.0
Loss after epoch 3: 498539.875
Loss after epoch 4: 467259.0
Loss after epoch 5: 457104.5
Loss after epoch 6: 448528.5
Loss after epoch 7: 440850.25
Loss after epoch 8: 418164.5
Loss after epoch 9: 395681.5
Loss after epoch 10: 392856.5
Loss after epoch 11: 389211.5
Loss after epoch 12: 386798.5
Loss after epoch 13: 384446.0
Loss after epoch 14: 383069.0
Loss after epoch 15: 377515.5
Loss after epoch 16: 378886.0
Loss after epoch 17: 376327.0
Loss after epoch 18: 374798.0
Loss after epoch 19: 352431.5
Loss after epoch 20: 333975.0
Loss after epoch 21: 333314.0
Loss after epoch 22: 329644.0
Loss after epoch 23: 328133.0
Loss after epoch 24: 328020.0
Loss after epoch 25: 324544.0
Loss after epoch 26: 323324.0
Loss after epoch 27: 322517.0
Loss after epoch 28: 320599.0
Loss after epoch 29: 319857.0
Loss after epoch 30: 318428.0
Loss after epoch 31: 316413.0
Loss after epoch 32: 315096.0
Loss after epoc

(55165041, 60602400)

### Procedemos a ensayar y a buscar relaciones de palabras en los documentos

In [22]:
#función para imprimir los resultados de skipgram y CBOW
def print_similar_words(skipgram, cbow ):
    # Convert to DataFrame
        df_compare = pd.DataFrame({
            "Skipgram": [f"{w} ({s:.3f})" for w, s in skipgram],
            "CBOW": [f"{w} ({s:.3f})" for w, s in cbow]
        })

        # Show as Markdown table
        from tabulate import tabulate
        print(tabulate(df_compare, headers="keys", tablefmt="github"))

In [24]:
# Palabras que MÁS se relacionan con...: (skigram vs CBOW)
skipgram = model_hp_potter.wv.most_similar(positive=["harry"], topn=10)
cbow = model_hp_potter_cbow.wv.most_similar(positive=["harry"], topn=10)
print_similar_words(skipgram, cbow )


|    | Skipgram           | CBOW             |
|----|--------------------|------------------|
|  0 | ron (0.784)        | ron (0.687)      |
|  1 | hermione (0.761)   | hermione (0.612) |
|  2 | back (0.611)       | hagrid (0.477)   |
|  3 | said (0.597)       | cho (0.474)      |
|  4 | ho (0.563)         | snape (0.457)    |
|  5 | hagrid (0.543)     | ginny (0.445)    |
|  6 | dumbledore (0.541) | quickly (0.433)  |
|  7 | pansys (0.534)     | back (0.428)     |
|  8 | harassed (0.527)   | neville (0.425)  |
|  9 | yeah (0.526)       | lupin (0.409)    |


### Observación:
- Es correcta la proximidad de Ron y Hermione a Harry en cbow, dado que los tres conforman el trio principal para toda la aventura. Esto refuerza el concepto que Cbow se enfoca en asociaciones directas, reflejadas en las relaciones de Harry
- Skipgram posee un foco sobre las palabras que dan contexto alrededor de la palabra target, lo que refleja una asociación fuerte con las emociones o acciones de, en este caso, el personaje principal.


In [25]:
# Palabras que MENOS se relacionan con...:
print_similar_words(model_hp_potter.wv.most_similar(negative=["magic"], topn=10), model_hp_potter_cbow.wv.most_similar(negative=["magic"], topn=10) ) 


|    | Skipgram             | CBOW                 |
|----|----------------------|----------------------|
|  0 | kettle (0.154)       | deliberately (0.414) |
|  1 | dancing (0.125)      | delight (0.381)      |
|  2 | lavender (0.125)     | lavender (0.350)     |
|  3 | filled (0.122)       | noisily (0.346)      |
|  4 | chos (0.120)         | hangings (0.341)     |
|  5 | mclaggen (0.114)     | wham (0.340)         |
|  6 | lump (0.108)         | gathered (0.339)     |
|  7 | glared (0.108)       | troy (0.337)         |
|  8 | inexplicably (0.106) | frantically (0.337)  |
|  9 | menacingly (0.104)   | entwined (0.329)     |


In [26]:
# Palabras que MÁS se relacionan con...:
print_similar_words(model_hp_potter.wv.most_similar(positive=["voldemort"], topn=10), model_hp_potter_cbow.wv.most_similar(positive=["voldemort"], topn=10) )

|    | Skipgram           | CBOW                |
|----|--------------------|---------------------|
|  0 | lord (0.786)       | voldemorts (0.670)  |
|  1 | voldemorts (0.673) | youknowwho (0.457)  |
|  2 | dumbledore (0.537) | faithful (0.447)    |
|  3 | pitiless (0.520)   | prophecy (0.443)    |
|  4 | hoodwinked (0.516) | nobody (0.434)      |
|  5 | wormtail (0.511)   | dumbledore (0.432)  |
|  6 | merciful (0.510)   | wormtail (0.429)    |
|  7 | conquer (0.503)    | youknowwhos (0.419) |
|  8 | faithful (0.502)   | snape (0.408)       |
|  9 | destroying (0.500) | hallows (0.401)     |


In [27]:
# Ensayar con una palabra que no está en el vocabulario:
model_hp_potter.wv.most_similar(negative=["flor"])

KeyError: "Key 'flor' not present in vocabulary"

In [28]:
# Palabras que MÁS se relacionan con...:
print_similar_words( model_hp_potter.wv.most_similar(positive=["station"], topn=10), model_hp_potter_cbow.wv.most_similar(positive=["station"], topn=10) )

|    | Skipgram          | CBOW              |
|----|-------------------|-------------------|
|  0 | cross (0.587)     | road (0.422)      |
|  1 | kings (0.565)     | kings (0.411)     |
|  2 | platform (0.514)  | town (0.379)      |
|  3 | charing (0.502)   | hogsmeade (0.371) |
|  4 | road (0.498)      | cobbled (0.363)   |
|  5 | twos (0.491)      | garden (0.355)    |
|  6 | doorstep (0.488)  | lane (0.351)      |
|  7 | bustle (0.482)    | charing (0.347)   |
|  8 | platforms (0.478) | bounding (0.347)  |
|  9 | london (0.473)    | owlery (0.346)    |


In [29]:
# el método `get_vector` permite obtener los vectores:
vector_potter = model_hp_potter.wv.get_vector("potter")
print(vector_potter)

[-0.0144272   0.05938755  0.05203345 -0.39078772  0.22168623 -0.28830454
  0.06266569  0.38287532 -0.4089668   0.21668044 -0.45236993  0.35154894
  0.22825393  0.18412986 -0.3883976  -0.10463033  0.77708906  0.26716182
 -0.47313967 -0.4485316   0.18065226  0.28218904  0.05533827 -0.30032116
 -0.06195628 -0.17393777 -0.10740775  0.29724106 -0.08157198 -0.12165734
 -0.3137081   0.09953272 -0.19349623  0.02229697 -0.12111918  0.53177065
 -0.32009712 -0.17851384 -0.08166971 -0.21678805  0.60667807  0.31987542
  0.19648829  0.06204046 -0.19431725 -0.04423736  0.26678056  0.32394195
  0.29124767  0.31853005  0.3135923   0.27029318  0.21145275 -0.38112223
  0.0284159   0.2261127  -0.3694094   0.41864222  0.12789497  0.2795231
 -0.5208892   0.43165505  0.28500116 -0.4440597  -0.26811352 -0.02778219
  0.30584773  0.29038617 -0.17952451  0.36685646 -0.3822276   0.00530026
  0.61005425  0.297594    0.3202067   0.3806472  -0.13455449 -0.38945192
  0.07230443  0.04184896  0.18631652  0.27131325 -0.

### Visualización de los vectores:

In [30]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [31]:
import plotly.io as pio
pio.renderers.default = "vscode"   # try this first
# pio.renderers.default = "browser"   # fallback → opens interactive chart in browser


In [32]:
# Graficar los embedddings en 2D
import pandas as pd
import plotly.express as px
import kaleido  # Necesario para guardar imágenes estáticas


MAX_WORDS = 200
# Reducir Skip-gram y CBOW a 3D
vecs_skip, labels_skip = reduce_dimensions(model_hp_potter, 2)
vecs_cbow, labels_cbow = reduce_dimensions(model_hp_potter_cbow, 2)

# DataFrame para Skip-gram
df_skip = pd.DataFrame({
    "x": vecs_skip[:MAX_WORDS, 0],
    "y": vecs_skip[:MAX_WORDS, 1],
    "word": labels_skip[:MAX_WORDS],
    "model": ["Skip-gram"] * MAX_WORDS
})

# DataFrame para CBOW
df_cbow = pd.DataFrame({
    "x": vecs_cbow[:MAX_WORDS, 0],
    "y": vecs_cbow[:MAX_WORDS, 1],
    "word": labels_cbow[:MAX_WORDS],
    "model": ["CBOW"] * MAX_WORDS
})

# Concatenanar ambos DataFrames
df_plot = pd.concat([df_skip, df_cbow])

# Scatter plot con color por modelo
fig = px.scatter(
    df_plot, x="x", y="y", text="word", color="model",
    title="Skip-gram vs CBOW Embeddings (proyección 2D)"
)

fig.update_traces(textposition="top center")  # move labels so they don’t overlap too much
fig.write_image("./images/Embeddings2d.png")   # static PNG

#### Embedding 2D:
![visualizacion del embedding en 2D](./images/Embeddings2d.png)

In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido  # Necesario para guardar imágenes estáticas

MAX_WORDS = 150

# Reducir Skip-gram y CBOW a 3D
vecs_skip, labels_skip = reduce_dimensions(model_hp_potter, 3)
vecs_cbow, labels_cbow = reduce_dimensions(model_hp_potter_cbow, 3)

#  DataFrame para Skip-gram
df_skip = pd.DataFrame({
    "x": vecs_skip[:MAX_WORDS, 0],
    "y": vecs_skip[:MAX_WORDS, 1],
    "z": vecs_skip[:MAX_WORDS, 2],
    "word": labels_skip[:MAX_WORDS]
})

#  DataFrame para CBOW
df_cbow = pd.DataFrame({
    "x": vecs_cbow[:MAX_WORDS, 0],
    "y": vecs_cbow[:MAX_WORDS, 1],
    "z": vecs_cbow[:MAX_WORDS, 2],
    "word": labels_cbow[:MAX_WORDS]
})

# Crear los subplots 3d
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=('Skip-gram Embeddings', 'CBOW Embeddings'),
    horizontal_spacing=0.1
)

# agregar  Skip-gram scatter plot
fig.add_trace(
    go.Scatter3d(
        x=df_skip["x"],
        y=df_skip["y"],
        z=df_skip["z"],
        mode='markers+text',
        text=df_skip["word"],
        textposition='middle center',
        marker=dict(size=3, color='blue'),
        name='Skip-gram'
    ),
    row=1, col=1
)

# agregar CBOW scatter plot
fig.add_trace(
    go.Scatter3d(
        x=df_cbow["x"],
        y=df_cbow["y"],
        z=df_cbow["z"],
        mode='markers+text',
        text=df_cbow["word"],
        textposition='middle center',
        marker=dict(size=3, color='red'),
        name='CBOW'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Skip-gram vs CBOW Word Embeddings (3D Projection)",
    title_x=0.5,
    showlegend=True,
    width=1400,
    height=700
)
fig.show()


fig.write_image("./images/Embeddings3d.png")   # static PNG

# PNG (static image - requires kaleido package)



#### Embedding 3D:
![visualizacion del embedding en 3D](./images/Embeddings3d.png)

### Conclusiones

El proceso de preprocesamiento y limpieza del dataset tuvo un impacto significativo en la calidad del texto utilizado para entrenar el modelo de *Word2Vec*.  

1. Reducción del vocabulario:
   - Tokens únicos antes de la limpieza: **2238**  
   - Tokens únicos después de la limpieza: **1628**  
   → Esto representa una reducción del **27%** en el tamaño del vocabulario, eliminando ruido y variaciones innecesarias (puntuación, mayúsculas, etc.).

2. Manejo de stopwords: 
   - Stopwords detectadas sin normalización: **6723**  
   - Stopwords detectadas después de la limpieza: **7053**  
   → La limpieza permitió una correcta identificación de stopwords, que inicialmente estaban ocultas por diferencias de formato (puntuación, capitalización).  

3. Estandarización del texto: 
   La conversión a minúsculas y la eliminación de caracteres no alfabéticos aseguraron que palabras semánticamente idénticas no fueran contadas como tokens diferentes.

4. Impacto en el modelo: 
   - El modelo de *Word2Vec* se entrenó sobre un corpus más consistente y menos ruidoso.  
   - Esto favorece que las representaciones vectoriales reflejen relaciones semánticas reales en lugar de artefactos de formato.
 En conclusión, el preprocesamiento no solo redujo el ruido en los datos, sino que también mejoró la representatividad semántica del corpus, logrando un vocabulario más compacto y útil para el entrenamiento de modelos de NLP.  

5. Relaciones entre palabras:
   - El modelo predice una relación correcta al elegir los nombres de personajes principales, o lugares de interés en los cuentos, lo que demuestra que se capturó adecuadamente el contexto de la obra.
     - Harry es asociado principalmente con sus mejores amigos
     - Voldemort con "lord" dado que siempre es referido como "Lord Voldermort" en las historias
     - Station, es referenciada a la estación en Londres donde se puede abordar el tren mágico rumbo a la escuela de magos.

6. Análisis de gráficos:
     - En el gráfico 2D se observan clusters de mayor cantidad de objetos de color azul, mientras que el rojo (Cbow) en general son puntos mas dispersos.
  